In [22]:
#HW 2
#1.Given 1000 records in a dataset, 1000 models are trained with 999 records as part of
#the training sample and the remaining 1 sample for testing, and the error rate is averaged out,
#this validation technique is called:
    #c. LOOCV

#2.In k-fold cross validation technique, the value of k being small could lead to which of
#the following in relation to the error rate
    #c. High Bias and Low variance 
    
#3.In k-fold cross validation technique, the value of k being large could lead to which of
#the following in relation to the error rate
    #b. Low bias and high variation
    
#4.Explain what regularization is and why it is useful
    #Regularization is the process finding ways to ajust ML models 
    #to reduce losses and prevent overfitting/underfitting. 

In [52]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

s3= boto3.resource('s3')
bucket_name= 'morgangant-bata-445-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'framingham.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [54]:
#Removing na values
heart= heart.dropna()

#scaling the data
scaler=MinMaxScaler()
heart_edit=pd.DataFrame(scaler.fit_transform(heart), index= heart.index, columns= heart.columns)
heart_edit

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1.0,0.184211,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.168378,0.106383,0.232804,0.277024,0.363636,0.104520,0.0
1,0.0,0.368421,0.333333,0.0,0.000000,0.0,0.0,0.0,0.0,0.281314,0.177305,0.349206,0.319680,0.515152,0.101695,0.0
2,1.0,0.421053,0.000000,1.0,0.285714,0.0,0.0,0.0,0.0,0.271047,0.208038,0.338624,0.237518,0.313131,0.084746,0.0
3,0.0,0.763158,0.666667,1.0,0.428571,0.0,0.0,1.0,0.0,0.229979,0.314421,0.497354,0.316045,0.212121,0.177966,1.0
4,0.0,0.368421,0.666667,1.0,0.328571,0.0,0.0,0.0,0.0,0.353183,0.219858,0.380952,0.183228,0.414141,0.127119,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4231,1.0,0.684211,0.666667,0.0,0.000000,0.0,0.0,1.0,0.0,0.151951,0.271868,0.349206,0.228308,0.363636,0.115819,0.0
4232,1.0,0.947368,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,0.129363,0.399527,0.518519,0.184198,0.161616,0.110169,1.0
4233,1.0,0.473684,0.000000,1.0,0.014286,0.0,0.0,1.0,0.0,0.410678,0.451537,0.465608,0.252787,0.222222,0.129944,1.0
4234,1.0,0.500000,0.666667,1.0,0.614286,0.0,0.0,0.0,0.0,0.193018,0.203310,0.338624,0.101066,0.212121,0.079096,0.0


In [55]:
#Defining input and target variables
x= heart_edit.drop(columns= ['TenYearCHD'], axis=1)
y= heart_edit['TenYearCHD']

#Defining the folds
kf=KFold(n_splits= 5, shuffle= True)

#Defining the lists to store results
f1score_1= list()
f1score_2= list()

for train_idx, val_idx in kf.split(x):
    #Splitting data
    x_train, x_val= x.iloc[train_idx], x.iloc[val_idx]
    y_train, y_val= y.iloc[train_idx], y.iloc[val_idx]
    
    #Model 1 
    x1= x_train[['age', 'currentSmoker', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']] 
    x_val_1= x_val[['age', 'currentSmoker', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']] 
    
    #Building the linear model
    md1= LogisticRegression().fit(x1, y_train)
    
    #Predicting on the validation set
    pred1= md1.predict(x_val_1)
    
    #Setting 25% threshold
    pre1=np.where(pred1*0.25,0,1)
    
    #Computing F1 score and storing it
    md1_f1= f1_score(y_val, pred1)
    f1score_1.append(md1_f1)
    
    
    #Model 2
    x2= x_train[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate', 'glucose']]
    x_val_2= x_val[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate', 'glucose']] 
    
    #Building the linear model
    md2= LogisticRegression().fit(x2, y_train)
    
    #Predicting on the validation set
    pred2= md2.predict(x_val_2)
     
    #Setting 25% threshold
    pre2=np.where(pred2*0.25,0,1)
    
    #Computing F1 score and storing it
    md2_f1= f1_score(y_val, pred2)
    f1score_2.append(md2_f1)

In [56]:
print(f1score_1)
print(f1score_2)

[0.06349206349206349, 0.09009009009009009, 0.0, 0.04724409448818898, 0.06896551724137931]
[0.03278688524590164, 0.01904761904761905, 0.0, 0.01652892561983471, 0.018348623853211007]


In [ ]:
#Based on the F1 scores from both models, Model 2 has higher F1 values so we would want to continue with Model 2 over Model 1!!